In [1]:
include("../LiPoSID.jl")

using HDF5
using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra
using Dates


In [2]:
function read_omega_and_decay_rates1(file, γᵢ)

    h5open(file,"r") do fid
        γ_group = open_group(fid, "gamma_"*γᵢ) # create coupling group   

        omega = read(γ_group["omega"])
        relaxation = read(γ_group["gamma_relaxation"])
        #de_phasing = read(γ_group["gamma_de_phasing"])
        #exitation = read(γ_group["gamma_exitation"])
        #de_polarisation = read(γ_group["gamma_de_polarisation"])

        return(omega, sqrt(relaxation)) #, de_phasing, de_polarisation)
        
    end
end

function absm(M::AbstractMatrix)
    sqrt(M'*M)
end

function my_fidelity(ρ, σ)
    F = (tr(absm(sqrt(ρ)*sqrt(σ))))^2 #matrix square root
    if imag(F) > 1e-3 throw(F, "Fidelity is complex number. Real part returned.") end
    real(F)
end



my_fidelity (generic function with 1 method)

In [3]:
data_dir = "../DATA/NO_CUT/"
println(data_dir)

models_dir = "../MODELS/"
tests_dir = "../TESTS/"

dodeca_files = ["State_D"*string(n) for n=1:20];

basis_files = ["State_B"*string(n) for n=1:4];

all_files = vcat(dodeca_files, basis_files)

test_files = all_files;

../DATA/NO_CUT/


In [4]:
model_file = "../TESTS/POP_LME_1_random_trn4_tst20_2024-Jan-21_at_16-18.h5"

gammas = ["0.079477", "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"];

date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

trajectories_file_name = "POP_LME_1_Bloch_NO_CUT_trajectories_"*date_and_time_string * ".h5"

for γᵢ in gammas

    println("γ =  "*γᵢ)

    w, r  = read_omega_and_decay_rates1(model_file, γᵢ)

    H = [ w   0
          0   0. ]

    J₁ = [ 0   r
           0   0. ]


    h5open(tests_dir*trajectories_file_name,"cw") do fid
        γ_group = create_group(fid, "gamma_"*γᵢ) # create coupling group           
    end

    for df in test_files # loop over initial states
        
        print(df*" ")

        ρₛ, tₛ = LiPoSID.get_rho_series(data_dir*df*"_CUT_data.h5", γᵢ) # LiPoSID.get_rho_series(data_dir*df*"_2CUT_data.h5", γᵢ)
        bᵗˢᵗ = LiPoSID.bloch(ρₛ)

        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)

        tᵗˢᵗ = convert(Vector{Float64}, tₛ)
        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]

        ρ₀ = DenseOperator(basis,Hermitian(ρₛ[1]))

        tˢⁱᵈ, ρˢⁱᵈ  = timeevolution.master(tᵗˢᵗ, ρ₀, DenseOperator(basis,H), [DenseOperator(basis,J₁)])

        bˢⁱᵈ = LiPoSID.bloch([ρₜ.data for ρₜ in ρˢⁱᵈ])

        Fᵐʸ = [my_fidelity(ρ₁, ρ₂.data) for (ρ₁, ρ₂) in zip(ρₛ, ρˢⁱᵈ)]

        Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈ)]    


        h5open(tests_dir*trajectories_file_name,"cw") do fid
            γ_group = open_group(fid, "gamma_"*γᵢ) # open coupling group
            init_state_group = create_group(γ_group, df) # create initial state group
            init_state_group["bloch_exact"] = convert.(Float64, bᵗˢᵗ)
            init_state_group["bloch_sid"] = convert.(Float64, bˢⁱᵈ) 
            init_state_group["fidelity"] = convert.(Float64, Fᴸᴹᴱₑₓ) 
            init_state_group["my_fidelity"] = convert.(Float64, Fᵐʸ) 
        end            
    
    end

    println()
end
println()
println(trajectories_file_name)

γ =  0.079477


State_D1 State_D2 

State_D3 State_D4 

State_D5 State_D6 State_D7 State_D8 

State_D9 State_D10 

...


γ =  251.33
State_D1 State_D2 State_D3 State_D4 State_D5 

State_D6 State_D7 State_D8 State_D9 State_D10 State_D11 

State_D12 State_D13 State_D14 State_D15 State_D16 State_D17 

State_D18 State_D19 State_D20 State_B1 State_B2 State_B3 

State_B4 

POP_LME_1_Bloch_NO_CUT_trajectories_2024-Jan-29_at_11-31.h5
